In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
training_data = datasets.FashionMNIST(
    root="torch_data",
    train=True,
    download=True,
    transform=ToTensor(),
    )

test_data = datasets.FashionMNIST(
    root="torch_data",
    train=False,
    download=True,
    transform=ToTensor(),
    )

test_size = int(0.5 * len(test_data))
val_size = len(test_data) - test_size
test_data, val_data = torch.utils.data.random_split(test_data, [test_size, val_size])

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
val_dataloader = DataLoader(val_data, batch_size=batch_size)
for X, y in val_dataloader:
    print("Shape of X [B, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape)
    break

Shape of X [B, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64])


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device}")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

using cuda
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
    print(f"Test Error: \n Accuracy {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.269379 [    0/60000]
loss: 0.357099 [ 6400/60000]
loss: 0.266227 [12800/60000]
loss: 0.384707 [19200/60000]
loss: 0.454057 [25600/60000]
loss: 0.402668 [32000/60000]
loss: 0.326905 [38400/60000]
loss: 0.461429 [44800/60000]
loss: 0.401715 [51200/60000]
loss: 0.443293 [57600/60000]
Test Error: 
 Accuracy 85.6%, Avg loss: 0.394080 

Epoch 2
-------------------------------
loss: 0.222263 [    0/60000]
loss: 0.320844 [ 6400/60000]
loss: 0.229753 [12800/60000]
loss: 0.326607 [19200/60000]
loss: 0.366031 [25600/60000]
loss: 0.369627 [32000/60000]
loss: 0.291078 [38400/60000]
loss: 0.456798 [44800/60000]
loss: 0.331737 [51200/60000]
loss: 0.371052 [57600/60000]
Test Error: 
 Accuracy 85.6%, Avg loss: 0.383425 

Epoch 3
-------------------------------
loss: 0.223342 [    0/60000]
loss: 0.309085 [ 6400/60000]
loss: 0.210466 [12800/60000]
loss: 0.280805 [19200/60000]
loss: 0.400015 [25600/60000]
loss: 0.329630 [32000/60000]
loss: 0.271408 [38400/60

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[10][0], test_data[10][1]
with torch.no_grad():
    pred = model(x.to(device))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sneaker", Actual: "Sneaker"


In [16]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

tensor([[1, 2],
        [3, 4]])

In [22]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.3716, 0.2955],
        [0.1582, 0.8651]]) 



In [28]:
plt.imshow(data, cmap='gray')

# Display the plot
plt.show()

device(type='cuda', index=0)